In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from efficientnet_pytorch import EfficientNet

from tqdm import tqdm
from torch.autograd import Variable
from PIL import Image


save_train_dir = 'E:\\UCMerced_LandUse\\train'
save_test_dir = 'E:/UCMerced_LandUse/test'


In [2]:
# train_transforms = torchvision.transforms.Compose([
#         torchvision.transforms.Resize((resize, resize), Image.BILINEAR),
#         torchvision.transforms.RandomHorizontalFlip(),
#         torchvision.transforms.RandomCrop(crop_size),
#         torchvision.transforms.ToTensor(),
#         torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
#                                          std=(0.229, 0.224, 0.225))
#     ])
# test_transforms = torchvision.transforms.Compose([
#         torchvision.transforms.Resize((resize, resize),Image.BILINEAR),
#         torchvision.transforms.CenterCrop(crop_size),
#         torchvision.transforms.ToTensor(),
#         torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
#                                          std=(0.229, 0.224, 0.225))
#     ])

In [3]:
resize = 224
crop_size = 224
def data_train(train_dataset):
    train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((resize, resize), Image.BILINEAR),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.RandomCrop(crop_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                         std=(0.229, 0.224, 0.225))
    ])
    train_data = torchvision.datasets.ImageFolder(root=train_dataset,
                                                  transform=train_transforms)
    CLASS = train_data.class_to_idx
    print('训练数据label与文件名的关系为:', CLASS)
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
    return train_data, train_loader
 
def data_test(test_dataset):
    test_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((resize, resize),Image.BILINEAR),
        torchvision.transforms.CenterCrop(crop_size),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                         std=(0.229, 0.224, 0.225))
    ])
    test_data = torchvision.datasets.ImageFolder(root=test_dataset,
                                                  transform=test_transforms)
    CLASS = test_data.class_to_idx
    print('验证数据label与文件名的关系为:', CLASS)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True)
    return test_data, test_loader

train_data, train_dataloader = data_train(save_train_dir)
test_data, test_dataloader = data_test(save_test_dir)


训练数据label与文件名的关系为: {'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}
验证数据label与文件名的关系为: {'agricultural': 0, 'airplane': 1, 'baseballdiamond': 2, 'beach': 3, 'buildings': 4, 'chaparral': 5, 'denseresidential': 6, 'forest': 7, 'freeway': 8, 'golfcourse': 9, 'harbor': 10, 'intersection': 11, 'mediumresidential': 12, 'mobilehomepark': 13, 'overpass': 14, 'parkinglot': 15, 'river': 16, 'runway': 17, 'sparseresidential': 18, 'storagetanks': 19, 'tenniscourt': 20}


In [4]:
data = iter(test_dataloader)
image ,target = data.next()
#Imshow(torchvision.utils.make_grid(image, padding=0))
print(target.size)
print(target.size(0))
print(target.numpy())


<built-in method size of Tensor object at 0x0000018636E92B88>
16
[ 6  7 20  0  5  2 20 15 12 19  8  2  7  2  0  0]


In [5]:
class Model(nn.Module):
    def __init__(self, num_classes=21):
        super(Model, self).__init__()
        self.backbone = EfficientNet.from_name("efficientnet-b4")
        self.backbone_pretrained = EfficientNet.from_pretrained("efficientnet-b4")
#         self.backbone = torchvision.models.resnet50(pretrained=True)
#         in_features = self.backbone.fc.in_features
        #self.g = nn.Sequential(nn.Linear(1000, 128),nn.ReLU(inplace=True) ,nn.Linear(512, 128))
        self.out = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        feature = torch.flatten(x, start_dim=1)
        output = self.out(feature)
        #print(x.shape)
        
        
        return output
    
model = Model()   

Loaded pretrained weights for efficientnet-b4


In [6]:
criterion = nn.CrossEntropyLoss()
"""
交叉熵损失函数，在函数费调用nn.functional()，会使用criterion(input, target)==(y_prediction, y)
"""
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
"""
声明了一个optmi类的对象，两个必要的参数是Adam(params, lr)
"""

'\n声明了一个optmi类的对象，两个必要的参数是Adam(params, lr)\n'

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
#model.to(device)

def train(model, data_loader, optimizer, batch_size):
    total_loss = 0.0
    total_num = 0
    train_bar = tqdm(data_loader)
    for image, target in train_bar:
        image, target = image.cuda(non_blocking=True), target.cuda(non_blocking=True)
        model.to(device)
        out = model(image)
        optimizer.zero_grad()
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        total_loss = total_loss + loss.item()
        total_num = total_num + batch_size
        train_bar.set_description("Train Epoch [{}/{}], loss = {:.4f} ".format(epoch, epochs, total_loss/total_num))   
        
def test(model, data_loader):
    total_num = 0
    accuracy = 0.0
    test_bar = tqdm(data_loader)
    with torch.no_grad():
        for image, target in test_bar:
            image, target = image.cuda(non_blocking=True), target.cuda(non_blocking=True)
            model.to(device)
            out = model(image)
            _, predicts = torch.max(out.data, 1)
            total_num = total_num + target.size(0)
            accuracy = accuracy + (target == predicts).sum().item()
            test_bar.set_description("Accuracy of the network on the 10000 test images: {:.4f}".format(100 * accuracy / total_num))
    

cuda:0


In [8]:
batch_size = 1
epochs = 30
for epoch in range(1,epochs+1):
    train(model, train_dataloader, optimizer, batch_size)
    test(model, test_dataloader)

Train Epoch [1/30], loss = 2.9731 : 100%|██████████████████████████████████████████████| 65/65 [00:31<00:00,  2.06it/s]
Accuracy of the network on the 10000 test images: 10.0382: 100%|███████████████████████| 66/66 [00:10<00:00,  6.41it/s]
Train Epoch [2/30], loss = 2.8817 : 100%|██████████████████████████████████████████████| 65/65 [00:29<00:00,  2.20it/s]
Accuracy of the network on the 10000 test images: 8.5086: 100%|████████████████████████| 66/66 [00:10<00:00,  6.40it/s]
Train Epoch [3/30], loss = 2.8318 : 100%|██████████████████████████████████████████████| 65/65 [00:29<00:00,  2.19it/s]
Accuracy of the network on the 10000 test images: 10.4167:  50%|███████████▌           | 33/66 [00:05<00:05,  6.32it/s]


KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()